<a href="https://colab.research.google.com/github/sainisatish/DeepLearningFromScratch/blob/main/Cat_And_Dog_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files

In [2]:
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"satishsaini","key":"6f61f9669d77c8a5654931b01aefdc8d"}'}

In [3]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [4]:
!kaggle competitions download -c dogs-vs-cats

 99% 806M/812M [00:03<00:00, 236MB/s]
100% 812M/812M [00:03<00:00, 244MB/s]


In [5]:
!unzip -qq dogs-vs-cats.zip

In [6]:
!unzip -qq train.zip

In [7]:
import os,shutil,pathlib

In [8]:
original_dir = pathlib.Path("train")
new_base_dir = pathlib.Path("cats_vs_dogs_small")

In [9]:
def make_subset(subset_name, start_index, end_index): 
    for category in ("cat", "dog"):
        dir = new_base_dir / subset_name / category
        os.makedirs(dir)
        fnames = [f"{category}.{i}.jpg" 
                  for i in range(start_index, end_index)]
        for fname in fnames:
            shutil.copyfile(src=original_dir / fname,
                            dst=dir / fname)

In [10]:
make_subset("train", start_index=0, end_index=1000)
make_subset("validation", start_index=1000, end_index=1500) 
make_subset("test", start_index=1500, end_index=2500)

In [11]:
from tensorflow import keras
from tensorflow.keras import layers

In [16]:
inputs = keras.Input(shape=(180,180,3))
x=layers.Rescaling(1./255)(inputs)
x=layers.Conv2D(filters=32,kernel_size=3,activation='relu')(x)
x=layers.MaxPooling2D(pool_size=2)(x)
x=layers.Conv2D(filters=64,kernel_size=3,activation='relu')(x)
x=layers.MaxPooling2D(pool_size=2)(x)
x=layers.Conv2D(filters=128,kernel_size=3,activation='relu')(x)
x=layers.MaxPooling2D(pool_size=2)(x)
x=layers.Conv2D(filters=256,kernel_size=3,activation='relu')(x)
x=layers.MaxPooling2D(pool_size=2)(x)
x=layers.Conv2D(filters=256,kernel_size=3,activation='relu')(x)
x=layers.Flatten()(x)
outputs=layers.Dense(1,activation='sigmoid')(x)
model = keras.Model(inputs=inputs,outputs=outputs)



In [17]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 180, 180, 3)]     0         
                                                                 
 rescaling (Rescaling)       (None, 180, 180, 3)       0         
                                                                 
 conv2d (Conv2D)             (None, 178, 178, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 89, 89, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 87, 87, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 43, 43, 64)       0         
 2D)                                                         

In [18]:
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics='accuracy')

In [19]:
from tensorflow.keras.utils import image_dataset_from_directory 

In [20]:
train_dataset=image_dataset_from_directory(new_base_dir/"train",
                                           image_size=(180,180),
                                           batch_size=32)

validation_dataset = image_dataset_from_directory(new_base_dir /"validation",
                                                  image_size=(180,180),
                                                  batch_size=32)

test_size = image_dataset_from_directory(new_base_dir /"test",
                                         image_size=(180,180),
                                         batch_size=32)

Found 2000 files belonging to 2 classes.
Found 1000 files belonging to 2 classes.
Found 2000 files belonging to 2 classes.


In [21]:
for data_batch,label_batch in train_dataset:
    print(f"data batch shape {data_batch.shape}")
    print(f"label batch shape { label_batch.shape}")
    break

data batch shape (32, 180, 180, 3)
label batch shape (32,)


In [22]:
callbacks = [keras.callbacks.ModelCheckpoint(filepath="convnet_from_scratch.keras",save_best_only=True,monitor="val_loss")]

In [23]:
history = model.fit(train_dataset,epochs=30,validation_data=validation_dataset,callbacks=callbacks)

Epoch 1/30
17/63 [=======>......................] - ETA: 1s - loss: 0.6954 - accuracy: 0.5147

KeyboardInterrupt: ignored

In [24]:
print(history.history.keys())

NameError: ignored

In [25]:
import matplotlib.pyplot as plt

In [26]:
accuracy,val_accuracy,loss,val_loss = history.history["accuracy"],history.history["val_accuracy"],history.history["loss"],history.history["val_loss"]

NameError: ignored

In [27]:
epochs = range(1,len(accuracy)+1)
plt.plot(epochs, accuracy, "bo", label="Training accuracy")
plt.plot(epochs, val_accuracy, "b", label="Validation accuracy")
plt.title("Training and validation accuracy")
plt.legend()
plt.figure()
plt.plot(epochs, loss, "bo", label="Training loss")
plt.plot(epochs, val_loss, "b", label="Validation loss")
plt.title("Training and validation loss")
plt.legend()
plt.show()

NameError: ignored

In [27]:
test_loss,test_acc = model.evaluate(test_size)

63/63 [==============================] - 4s 43ms/step - loss: 2.9484 - accuracy: 0.6895


In [29]:
data_augmentation = keras.Sequential(
    [
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(0.1),
        layers.RandomZoom(0.2),
    ]
    )


In [32]:
inputs = keras.Input(shape=(180,180,3))
x= data_augmentation(inputs)
x= layers.Rescaling(1./255)(x)
x= layers.Conv2D(filters=32,kernel_size=3,activation='relu')(x)
x= layers.MaxPooling2D(pool_size=2)(x)
x= layers.Conv2D(filters=64,kernel_size=3,activation='relu')(x)
x= layers.MaxPooling2D(pool_size=2)(x)
x= layers.Conv2D(filters=128,kernel_size=3,activation='relu')(x)
x= layers.MaxPooling2D(pool_size=2)(x)
x= layers.Conv2D(filters=256,kernel_size=3,activation='relu')(x)
x= layers.MaxPooling2D(pool_size=2)(x)
x= layers.Conv2D(filters=256,kernel_size=3,activation='relu')(x)
x= layers.Flatten()(x)
x= layers.Dropout(0.5)(x)
outputs= layers.Dense(1,activation='sigmoid')(x)
model_aug = keras.Model(inputs=inputs,outputs=outputs)

In [33]:
model_aug.compile(optimizer="rmsprop",loss='binary_crossentropy',metrics=["accuracy"])

In [34]:
model_aug.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 180, 180, 3)]     0         
                                                                 
 sequential_2 (Sequential)   (None, 180, 180, 3)       0         
                                                                 
 rescaling_1 (Rescaling)     (None, 180, 180, 3)       0         
                                                                 
 conv2d_5 (Conv2D)           (None, 178, 178, 32)      896       
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 89, 89, 32)       0         
 2D)                                                             
                                                                 
 conv2d_6 (Conv2D)           (None, 87, 87, 64)        18496     
                                                           

In [36]:
callbacks = [keras.callbacks.ModelCheckpoint(filepath="new_model_with_aug.keras"),save_best_only=True,monitor="val_loss"]

SyntaxError: ignored